In [28]:
import spacy
import snscrape.modules.twitter as sntwitter
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk

In [29]:
def strip_emoji(text):
        emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)
        return emoji_pattern.sub('EMOJI', text)

In [30]:
def extractor(query: str, csvname: str):
    tweets = []

    for tweet in sntwitter.TwitterSearchScraper(query=query).get_items():
        tweets.append(
            [
                tweet.date,
                tweet.id,
                tweet.content,
                tweet.user.username,
            ]
        )
    df = pd.DataFrame(tweets, columns=["Datetime", "Tweet Id", "Text", "Username"])
    df["no_http"] = [re.sub(r"http\S+", "", tweet) for tweet in df["Text"]]
    df["no_arroba"] = [re.sub(r"[@]", "", x) for x in df["no_http"]]
    df["no_hastag"] = [re.sub(r"#", "", x) for x in df["no_arroba"]]
    df["no_espaço"] = [re.sub(r"\n|\t|\r", " ", x) for x in df["no_hastag"]]
    df['texto_processado'] = df['no_espaço'].apply(strip_emoji)

    df.to_parquet(csvname,engine='fastparquet',compression='gzip')


In [31]:
extractor(
    "Vasco lang:pt until:2022-11-06 since:2022-11-05",
    "data_vasco.parquet",
)

In [ ]:
df = pd.read_parquet("data_vasco.parquet")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12964 entries, 0 to 12963
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype              
---  ------          --------------  -----              
 0   Datetime        12964 non-null  datetime64[ns, UTC]
 1   Tweet Id        12964 non-null  int64              
 2   Text            12964 non-null  object             
 3   Username        12964 non-null  object             
 4   Processed Text  12964 non-null  object             
dtypes: datetime64[ns, UTC](1), int64(1), object(3)
memory usage: 506.5+ KB


In [ ]:
df.head(30)

,Datetime,Tweet Id,Text,Username,Processed Text,Emoji Tag
0,2022-11-05 23:59:48+00:00,1589044844046864386,@FCPorto A frase abaixo dita por Cyro Aranha g...,Doraroxo23,FCPorto A frase abaixo dita por Cyro Aranha gr...,FCPorto A frase abaixo dita por Cyro Aranha gr...
1,2022-11-05 23:59:41+00:00,1589044815194652673,"Na moral, estou a todo instante pensando na me...",WesleyL52035066,"Na moral, estou a todo instante pensando na me...","Na moral, estou a todo instante pensando na me..."
2,2022-11-05 23:59:33+00:00,1589044780054777856,"Pessoal, amanhã, às 15h, tem @VascodaGama e @F...",LMC_jornalista,"Pessoal, amanhã, às 15h, tem VascodaGama e Fla...","Pessoal, amanhã, às 15h, tem VascodaGama e Fla..."
3,2022-11-05 23:59:31+00:00,1589044772949618688,@CristiFreitasS @TNTSportsBR Irmão quando você...,yuriikelviinn11,CristiFreitasS TNTSportsBR Irmão quando vocês ...,CristiFreitasS TNTSportsBR Irmão quando vocês ...
4,2022-11-05 23:59:24+00:00,1589044744931667968,@yesbruno @WandersonLucin8 @FabianoAlvesC @Rec...,LuanLessa14,yesbruno WandersonLucin8 FabianoAlvesC Recibo_...,yesbruno WandersonLucin8 FabianoAlvesC Recibo_...
5,2022-11-05 23:59:20+00:00,1589044728632573952,"@pedrosa Fala, Pedrosa! saiu a provável escala...",raaycunnha,"pedrosa Fala, Pedrosa! saiu a provável escalaç...","pedrosa Fala, Pedrosa! saiu a provável escalaç..."
6,2022-11-05 23:59:20+00:00,1589044725134557184,@wesley_curty Vasco é o clube que mais incomod...,ShalomA88725111,wesley_curty Vasco é o clube que mais incomoda...,wesley_curty Vasco é o clube que mais incomoda...
7,2022-11-05 23:59:17+00:00,1589044714677751809,@animefla ceara por não ter rebaixado o vasco,arrscaetta,animefla ceara por não ter rebaixado o vasco,animefla ceara por não ter rebaixado o vasco
8,2022-11-05 23:59:15+00:00,1589044703546470400,"Me trocou por um otário, que ainda mais torce ...",betinho_png,"Me trocou por um otário, que ainda mais torce ...","Me trocou por um otário, que ainda mais torce ..."
9,2022-11-05 23:59:14+00:00,1589044702237536256,supercopa -&gt; final da luca -&gt; vasco da g...,daanielt82,supercopa -&gt; final da luca -&gt; vasco da g...,supercopa -&gt; final da luca -&gt; vasco da g...
